### First part

In [37]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import List
from benchmark.benchmarkFunctions import StybliskiTang
import time


import numpy as np

def nelder_mead(fun, x0 = None, step=0.1, low=-5, high=5, no_improve_thr=10e-6, no_improv_break=10, max_iter=10, alpha=1., gamma=2., rho=-0.5, sigma=0.5):
    """
    Nelder mead optimization algorithm

    Parameters
    - f (function): function to optimize, must return a scalar score and operate over a numpy array of the same dimensions as x_start
    - x_start (numpy array): initial position
    - step (float): look-around radius in initial step
    - no_improv_thr,  no_improv_break (float, int): break after no_improv_break iterations with
            an improvement lower than no_improv_thr
    - max_iter (int): always break after this number of iterations.
            Set it to 0 to loop indefinitely.
    - alpha, gamma, rho, sigma (floats): parameters of the algorithm (see Wikipedia page for reference)
    
    Returns: 
    - A tuple (best parameter array, best score)
    """

    if(x0 is None): x0 = np.random.uniform(low=low, high=high, size=(1,2))

    #dim = len(x_start)
    X = [x0.copy()]
    Y = [fun(x0)]
    #no_improv = 0

    # Apply the displacement in each direction, one at a time
    for i in range(x0.shape[1]):
        tmp = x0.copy()
        tmp[0,i] += step
        X.append(tmp)
        Y.append(fun(X[-1]))

    for iteration in range(max_iter):
        
        # Get the best value
        best = np.min(Y)
        
        # break after no_improv_break iterations with no improvement
        #print '...best so far:', best

        if (best < prev_best - no_improve_thr):
            no_improv = 0
            prev_best = best
        else:
            no_improv += 1

        if no_improv >= no_improv_break: break

        # centroid
        xt = np.zeros(1, x0.shape[1])
        for x in X:
            for i, coord in enumerate(x):
                xt[i] = coord/(len(res)-1)

        # reflection
        xr = xt + alpha*(xt - X[-1])
        rscore = fun(xr)
        if (res[0][1] <= rscore < res[-2][1]):
            del res[-1]
            res.append([xr, rscore])
            continue

        # expansion
        if (rscore < res[0][1]):
            xe = x0 + gamma*(x0 - res[-1][0])
            escore = f(xe)
            if escore < rscore:
                del res[-1]
                res.append([xe, escore])
                continue
            else:
                del res[-1]
                res.append([xr, rscore])
                continue

        # contraction
        xc = xt + rho*(xt - res[-1][0])
        cscore = fun(xc)
        if cscore < res[-1][1]:
            del res[-1]
            res.append([xc, cscore])
            continue

        # reduction
        x1 = res[0][0]
        nres = []
        for tup in res:
            redx = x1 + sigma*(tup[0] - x1)
            score = f(redx)
            nres.append([redx, score])
        res = nres



nelder_mead(StybliskiTang, low=-5,high=5)

-74.97246103115883
-74.97246103115883
-74.97246103115883
-74.97246103115883
-74.97246103115883
-74.97246103115883
-74.97246103115883
-74.97246103115883
-74.97246103115883
-74.97246103115883


In [72]:
import numpy as np




x0 = np.array([2,2]).reshape(-1,1)
central_composite(x0, 1)


array([[2, 3, 1, 3, 1],
       [2, 3, 3, 1, 1]])

### Second part

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from typing import List
from scipy.stats import norm

def StybliskiTang(x: List[float]) -> float:
    f, dimension = 0, len(x)

    for i in range(dimension):
        xi = x[i]
        f += xi**4 - 16*xi**2 + 5*xi
    return f


def surrogate_model(model: GaussianProcessRegressor, X):
    prediction = model.predict(X, return_std=True)
    return prediction

def acquisition(X, Xsamples, model):
    yhat, _ = surrogate_model(model, X)
   
    best    = np.max(yhat)
    print("best; ", best)
    mu, std = surrogate_model(model, Xsamples)
    print(yhat)
    print(mu-np.max(yhat))
    probs = norm.cdf((mu - best) / (std + 1e-9))
    print(probs)
    return probs

def opt_acquisition(X, model):
    # Decide new samples to analyze
    Xsamples =  np.random.uniform(low=-5, high=5, size=[15,2])

    scores = acquisition(X, Xsamples, model)

    ix = np.argmax(scores)
    return Xsamples[ix,0]

# Define the number of samples
n_samples = 15

# GaussianProcessRegressor(kernel=kernel)
model = GaussianProcessRegressor()
# Pick random sample values
X = np.random.uniform(low=-5, high=5, size=[n_samples,2])
print(X.shape)
# Compute the expensive objective function (in the wind tunnel)
Y = StybliskiTang(X.T)

# Apply the Bayesian optimization, namely refit the model
model.fit(X,Y)

# Estimate the surrogate mdel values
y, _ = surrogate_model(model, X)


# Perform the optimization process
for i in range(100):
    # select the next point to sample (random chosen point with the max likelihood)
    x = opt_acquisition(X, model)
    print(x.shape)
    # sample the point (EXPENSIVE FUNCTION)
    actual = StybliskiTang(x)
    print(actual.shape)
    # SURROGATE VALUES
    est, _ = surrogate_model(model, x)

    print('>x=%.3f, f()=%3f, actual=%.3f' % (x, est, actual))

    # add the data to the dataset FOR THE MODEL
    X = np.vstack((X, [[x]]))
    y = np.vstack((y, [[actual]]))

    # update the model
    model.fit(X, Y)
 

# best result
ix = np.argmax(Y)
print('Best Result: x=%.3f, y=%.3f. The estimated value is %.3f' % (X[ix], Y[ix], y[ix]))